In [ ]:
from openai import OpenAI
from typing import Any
import IPython.display as ipd
from aimo2.utils import mdlatex
from copy import deepcopy
from aimo2.parser import extract_boxed_text, latex_to_int
import matplotlib.pyplot as plt
from typing import Optional
import numpy as np
from transformers import AutoTokenizer
import time
import re

In [ ]:
model = "casperhansen/deepseek-r1-distill-qwen-1.5b-awq"
model = "evanarlian/DeepScaleR-1.5B-Preview-AWQ"
client = OpenAI(api_key="-", base_url="http://localhost:8000/v1")
tokenizer = AutoTokenizer.from_pretrained(model)

In [ ]:
history: Any = [
    # {
    #     "role": "system",
    #     # "content": "Be very concise, terse, to the point, confident, and put final answer in \\boxed{}.",
    #     "content": "Be concise and put final answer in \\boxed{}.",
    #     # "content": "The system rewards shorter answer. Be very concise, terse, to the point, confident, and put final answer in \\boxed{}.",
    #     # "content": "Write down the related math theorems first, then think, then put final answer in \\boxed{}.",
    #     # "content": "Give me the simplest solution, then put final answer in \\boxed{}.",
    # },
    # {
    #     "role": "user",
    #     "content": "Three airline companies operate flights from Dodola island. Each company has a different schedule of departures. The first company departs every 100 days, the second every 120 days and the third every 150 days. What is the greatest positive integer $d$ for which it is true that there will be $d$ consecutive days without a flight from Dodola island, regardless of the departure times of the various airlines?",
    # },
    {
        "role": "user",
        "content": "What is the first prime number?",
    },
]
t0 = time.perf_counter()
completion = client.chat.completions.create(
    model=model,
    messages=history,
    temperature=0.6,
    logprobs=True,
    # top_p=0.9,
    # max_tokens=300,
    # stop=["</think>"],
    extra_body={
        "min_p": 0.1,
    },
)
elapsed = time.perf_counter() - t0
reply = completion.choices[0].message.content
logprobs = np.array([l.logprob for l in completion.choices[0].logprobs.content])
history.append({"role": "assistant", "content": reply})
toks = completion.usage.completion_tokens
tok_per_sec = toks / elapsed
print(toks, tok_per_sec, logprobs.mean(), logprobs.std())  # 475-485
mdlatex(reply)

In [ ]:
extract_boxed_text(reply)

In [ ]:
plt.plot(np.sort(logprobs))

In [ ]:
plt.plot(logprobs)

In [ ]:
completion.choices[0].logprobs.content

In [ ]:
# in case the model does not make the final answer
# use hacks because </think> crashes the apply_chat_template
fake_history = deepcopy(history)
fake_history[-1]["content"] = (
    fake_history[-1]["content"].replace("</think>", "<think2>")
    + "\n\nI repeat, the final answer is: \\[\n\\boxed{"
)
prompt = tokenizer.apply_chat_template(
    fake_history,
    tokenize=False,
    add_generation_prompt=False,
    continue_final_message=True,
).replace("<think2>", "</think>")
completion = client.completions.create(
    model=model,
    prompt=prompt,
    temperature=0.6,
    # max_tokens=10000,
)
reply = fake_history[-1]["content"] + completion.choices[0].text
# history.append({"assistant": reply})
mdlatex(reply)

In [ ]:
history

In [ ]:
reply

In [ ]:
tokenizer.tokenize("\\[\n\\boxed{\\dfrac{5}{6}}\n\\]")

In [ ]:
def extract_boxed_text(text: str) -> Optional[int]:
    pattern = r"\boxed{(.+?)}"
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    for match in matches[::-1]:
        if match != "":
            return match
    return ""
extract_boxed_text()

In [ ]:
text = "Final answer: \\boxed{hello123/sddf}"
pattern = r"\boxed{(.+?)}"
matches = re.findall(pattern, text)
if not matches:
    return ""
for match in matches[::-1]:
    if match != "":
        return match
return ""

In [ ]:
-900 % 1000

# Answer parsing
* extract from \\boxed{..}
* check if that is computable 

In [ ]:
def extract_boxed_content(latex_str):
    stack = []
    result = []
    i = 0
    while i < len(latex_str):
        if latex_str[i:].startswith("\\boxed{"):
            i += 7  # Skip "\boxed{"
            start = i
            stack = [1]
            while i < len(latex_str) and stack:
                if latex_str[i] == "{":
                    stack.append(1)
                elif latex_str[i] == "}":
                    stack.pop()
                i += 1
            if not stack:
                result.append(latex_str[start : i - 1])
        else:
            i += 1
    return result




# Example
latex_output = r"The answer is \(\boxed{\frac{3}{2} + \sqrt{16}}\)"
boxed_content = extract_boxed_content2(latex_output)
print(boxed_content)  # Outputs: \frac{3}{2} + \sqrt{16}
print("=============")
print(extract_boxed_content2("ss"))  # Outputs: \frac{3}{2} + \sqrt{16}

In [ ]:
extract_boxed_content(r"\(\boxed{\frac{3}{2}}\)")